In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections, Machine

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors import ChargeWeighter, Averager, Bypass, Lowpass, Sinc, NoiseGenerator
from PyHEADTAIL.feedback.digital_processors import HilbertPhaseShiftRegister, ADC, DAC, FIR_Filter
from PyHEADTAIL.particles.slicing import UniformBinSlicer

np.random.seed(0)

In [ ]:
""" 
    In this example, a model for a more realistic feedback system is presented. Note that it does not try to 
    emulate any system from the real life, but different elements required by more realistic simulations are
    presented. The simulations might very sensitive to details of the feedback model and it must be remembered
    that this is a toy model!
    
    The example is directly based on the example including one pickup and one kicker in the file 
    002_separated_pickup_and_kicker.ipynb. This example focuses on the details of the signal processors, and 
    more details about other parts of the model can be found from there.

    @author Jani Komppula
    @date 26/09/2016
    @copyright CERN
"""

In [ ]:
# Basic parameters for the simulation
n_macroparticles = 10000
n_slices = 200
n_segments = 5
n_sigma_z = 3 

n_turns = 2000

# Longitudinal motion of the bunch has not been taken into account in this example.
machine = Machine(Q_s = 0.00000001)
# If you want to include the longitudinal motion, please uncomment the following line
# machine = Machine(Q_s = 0.0020443)

bunch_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles,n_segments, 
                                                             n_slices,n_sigma_z)
bunch_unkicked = copy.deepcopy(bunch_ref)


xp_per_x = bunch_unkicked.sigma_xp()/bunch_unkicked.sigma_x()
yp_per_y = bunch_unkicked.sigma_yp()/bunch_unkicked.sigma_y()

In [ ]:
# This creates an artificially kicked bunch, which will be damped with different feedback systems

f_kick = 1e9 # Frequency of the kick [Hz]
kick_amplitude = 0.003 # [m]

slicer_for_kicks = UniformBinSlicer(n_slices=200, n_sigma_z=3)
slice_set = bunch_ref.get_slices(slicer_for_kicks, statistics=True)
z = np.array(slice_set.mean_z)
p_idx = slice_set.particles_within_cuts
s_idx = slice_set.slice_index_of_particle.take(p_idx)

kick_x = kick_amplitude*np.sin(2.*pi*f_kick*z/c)
kick_xp = xp_per_x*kick_amplitude*np.sin(2.*pi*f_kick*z/c)
kick_y = kick_amplitude*np.sin(2.*pi*f_kick*z/c)
kick_yp = yp_per_y*kick_amplitude*np.sin(2.*pi*f_kick*z/c)

bunch_ref.x[p_idx] += kick_x[s_idx]
bunch_ref.xp[p_idx] += kick_xp[s_idx]
bunch_ref.y[p_idx] += kick_y[s_idx]
bunch_ref.yp[p_idx] += kick_yp[s_idx]
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map] + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

In [ ]:

feedback_gain = 0.033
# feedback_gain = (0.01,0.01)


# Parameters for the feedback system
pickup_bandwidth = 2e9 # [Hz]
n_FIT_taps = 5
FIR_bandwidth = 0.8e9 # [Hz] 
kicker_bandwidth = 0.8e9 # [Hz]

# Parameters for the ADC and the DAC
sampling_rate = 4e9 # [Hz]
ADC_n_bits = 12
ADC_range = (-3e-3, 3e-3) # in the units of displacement in the pickup [m]

noise_level = 1.
noise_level_x = noise_level * bunch_unkicked.sigma_x()
noise_level_y = noise_level * bunch_unkicked.sigma_y()

# Parameters for the registers
delay = 1 
n_values = 7

In [ ]:
# A reference feedback system for the examples presented in this file is the ideal bunch feedback:

bunch_OneBox_bunch = copy.deepcopy(bunch_ref)
tracker_OneBox_bunch = BunchTracker(bunch_OneBox_bunch)
slicer_OneBox_bunch = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'average_weight'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_OneBox_bunch,processors_bunch_x,processors_bunch_y)
total_map_OneBox_bunch = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_OneBox_bunch,total_map_OneBox_bunch,tracker_OneBox_bunch)

# That feedback system is extended to consist of a separated pickup and a kicker in this file. If you don't
# understand details of the code above, please study examples from the file 001_ideal_feedbacks.ipynb

In [ ]:
bunch_separated_example = copy.deepcopy(bunch_ref)
tracker_separated_example = BunchTracker(bunch_separated_example)
slicer_separated_example = copy.deepcopy(slicer_ref)

# The signal processing of the transfer feedback system is often divided into three different sections:
#
# Front end: 
# ----------
# The signal path from the pickup plates to the analog to digital converter (ADC) is called by 
# the Fron End. If the direct signal from the pickup plates is used, it is proportional to the charge weighted
# positions of the slices. Thus, ChargeWeighter() must be used. The limited bandwidth of the pickup can be 
# implemented by using filters. The simples model is a RC-lowpass filter (Lowpass(...)). 
# The pickup has typically significantly higher bandwidth than the other parts of the system, 
# i.e. it is not significantly limiting the bandwidth of the feedback system.
#
# Every part of the system induces noise to the signal. In this example, noise have been added only after the
# pickup, but it could be added after every processor. Note that the filters after the noise generator 
# attenuates the noise and a noise generator for specific noise spectrums has not been (yet) implemented.
#
# In some real life systems, the readings from the pickup plates are normalized to absolute position values 
# by by calculating a ratio of sum and difference signals from the plates (Delta/Sigma method). The simplest
# way to model this method is to remove ChargeWeighter() from the list of signal processors. For the more 
# accurate model, a special signal processor is required, which can be found from 
# https://gitlab.cern.ch/jakomppu/PyHEADTAIL_feedback.git
#
# Digital signal processing (DSP): 
# --------------------------------
# In many systems, the signal is mainly processed digitally. This means that not only the signal over the 
# bunch might be filttered (e.g. Low pass FIR filter), but also the signals from the same bunch over multiple
# turns can be processed (e.g. betatron phase shift, notch filtering). The finite simpling rate and accuracy 
# of the dac can be modelled by using an ADC and a DAC. Betatron phase shift can be modelled by using register,
# e.g. HilbertPhaseShiftRegister.
# 
# Back end:
# ---------
# The signal processing after the DAC is called by the Back End. The key components for this part are an
# analog filter used for reducing ripple caused by DAC, a power amplifier and a kicker. Typically the power 
# amplifier or the kicker are the limiting elements in terms of bandwidth. Frequency response of the power
# amplifier might be (partially) phase linearized by using digital signal processing, or frequency response of the 
# amplifier might decay shortly after the cutoff frequency. Thus, any of the lowpass filters might be
# a good model, but Sinc filter could be the safest option.
#
#
# In the simulations these three sections of signal processing are divided into the signal 
# processors in the pickups and the kickers. This means that the Front end is completely modelled in
# the pickup and the Back end is completely modelled in the Kicker.
# 
# The digital signal processing can be modelled completely in the pickup or it can be separated into 
# the pickup and the kicker. The trick is that the last element of the list of the signal processors in 
# the pickup must be a register. Thus, if the register is digital, i.e. the DAC should be after the register 
# in the list of signal processors in the kicker. However, in practice, the register can be after the DAC, if 
# the DAC is only a resampler and betatron phase shifting is a linear process in the register

processors_separated_pickup_x = [
    ChargeWeighter(), # Front end
    Lowpass(pickup_bandwidth), # Front end
    NoiseGenerator(noise_level_x), # Front end
    ADC(sampling_rate), # Digital signal processing
    FIR_Filter(n_FIT_taps,FIR_bandwidth,sampling_rate),# Digital signal processing
    DAC(sampling_rate),# Digital signal processing
    HilbertPhaseShiftRegister(n_values, machine.Q_x, delay)# Digital signal processing
]
processors_separated_pickup_y = [
    ChargeWeighter(),
    Lowpass(pickup_bandwidth),
    NoiseGenerator(noise_level_y),
    ADC(sampling_rate,ADC_n_bits,ADC_range),
    FIR_Filter(n_FIT_taps,FIR_bandwidth,sampling_rate),
    DAC(sampling_rate),
    HilbertPhaseShiftRegister(n_values, machine.Q_y, delay)
]

pickup_position_x = 1.*2.*pi/float(n_segments)*machine.Q_x
pickup_position_y = 1.*2.*pi/float(n_segments)*machine.Q_y

pickup_map = PickUp(slicer_separated_example,processors_separated_pickup_x,processors_separated_pickup_y, 
       pickup_position_x, pickup_position_y)

processors_separated_kicker_x = [Sinc(kicker_bandwidth)]
processors_separated_kicker_y = [Sinc(kicker_bandwidth)]


# Rememeber to change the index, if the number of processors is changed in the processors list of the pickup
registers_x = [processors_separated_pickup_x[-1]] # Back end
registers_y = [processors_separated_pickup_y[-1]]

kicker_position_x = 2.*2.*pi/float(n_segments)*machine.Q_x
kicker_position_y = 2.*2.*pi/float(n_segments)*machine.Q_y

kicker_map = Kicker(feedback_gain, slicer_separated_example,
                    processors_separated_kicker_x, processors_separated_kicker_y,
                    kicker_position_x, kicker_position_y,
                    registers_x, registers_y, xp_per_x, yp_per_y)

total_map_separated_example = [trans_map[0]] + [pickup_map] + [trans_map[1]] + [kicker_map]
for element in trans_map[2:]:
    total_map_separated_example += [element]
total_map_separated_example += [long_map]
    
    
track(n_turns, bunch_separated_example,total_map_separated_example,tracker_separated_example)

In [ ]:
# If details of the feedback system and/or the sensitivity of the model for those details is not known, it 
# might be beter to use a much simpler and stabler model for the feedback system. One options is Sinc filter,
# but it is good to remember that Sinc filter is not an ideal as presented in the previous example 
# (004_analog_signal_processors.ipynb).

bunch_simplest_model = copy.deepcopy(bunch_ref)
tracker_simplest_model = BunchTracker(bunch_simplest_model)
slicer_simplest_model = copy.deepcopy(slicer_ref)

processors_simplest_model_x = [
    ChargeWeighter(),
    Sinc(kicker_bandwidth)
]
processors_simplest_model_y = [
    ChargeWeighter(),
    Sinc(kicker_bandwidth)
]

feedback_map = OneboxFeedback(feedback_gain,slicer_simplest_model,
                              processors_simplest_model_x,processors_simplest_model_y)
total_map_simplest_model = [i for i in trans_map] + [long_map] + [feedback_map]

track(n_turns, bunch_simplest_model,total_map_simplest_model,tracker_simplest_model)

In [ ]:
compare_traces([tracker_OneBox_bunch,tracker_separated_example,tracker_simplest_model],
               ['Ideal bunch', 'Complex model', 'Simplest model'])
compare_projections([bunch_OneBox_bunch, bunch_separated_example,  bunch_simplest_model,bunch_ref], 
                    ['Ideal bunch', 'Complex model', 'Simplest model','Before damping'])